In [1]:
from process.preproc_clinic import writeClinic
from process.preproc_img import imgRedir, mvRaw, preprocFSL, preprocCAT12
from process.preproc_combine import genHCData, combinePatData
from src.feature.base import genROI
from src.utils.data import getPandas, getConfig, writeConfig
from src.model.stats import stats_analyze
import os

In [2]:
# Root directory as working directory
os.chdir('..')
os.getcwd()

'/home/biobot/disk/ldopa'

In [ ]:
# Get Image Data
imgRedir()
# Get Clinical Data
writeClinic()
# Combine Data & Generate HC Data
combinePatData()
genHCData()
# Generate Subject Directory & Move Raw Image Data
mvRaw(getPandas('img_raw'))
# Now all raw image data and clinical data are set up

In [ ]:
# Image Preprocessing
# For Patient Data:
preprocFSL('pat_data')
preprocCAT12('pat_data')
# For HC Data:
preprocFSL('hc_data')
preprocCAT12('hc_data')

# Time Consuming!!!

In [3]:
# Dataset Split
# One patient may have multiple images, so we need to split the dataset by patient
from sklearn.model_selection import GroupShuffleSplit

pat = getPandas('pat_data')
data_conf = getConfig('data')

splitter = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
split = splitter.split(pat, groups=pat['PATNO'])
train_inds, test_inds = next(split)

In [4]:
# Data Analysis
group = data_conf['data_group']
x = pat[group['demo'] + group['clinic'] + ['PATNO']]
y = pat[['CAT_MDS']]
x_clinic_train = x.iloc[train_inds].reset_index(drop=True)
x_clinic_test = x.iloc[test_inds].reset_index(drop=True)
y_train = y.iloc[train_inds].reset_index(drop=True)
y_test = y.iloc[test_inds].reset_index(drop=True)

stats_analyze(x_clinic_train, x_clinic_test, y_train, y_test, data_conf)

SEX chi2 p: 0.6017923111110748
AGE Normaltest p_train: [0.03305021], p_test: [0.25936315]
AGE ranksums p: RanksumsResult(statistic=array([-1.08030697]), pvalue=array([0.28000551]))
NUPDR3OF Normaltest p_train: [0.19255173], p_test: [0.04134491]
NUPDR3OF ranksums p: RanksumsResult(statistic=array([-0.77829997]), pvalue=array([0.43639219]))
LEDD Normaltest p_train: [3.21029424e-11], p_test: [0.00100613]
LEDD ranksums p: RanksumsResult(statistic=array([-0.66920493]), pvalue=array([0.50336476]))
DURATION Normaltest p_train: [0.00115343], p_test: [0.47174464]
DURATION ranksums p: RanksumsResult(statistic=array([-1.09893295]), pvalue=array([0.27179731]))



In [5]:
# Save train/test indices to data config
data_conf['indices'] = {
    'train': train_inds.tolist(),
    'test': test_inds.tolist()
}
writeConfig('data', data_conf)

## Feature Extraction

In [ ]:
## Texture Features
# Generate ROI
genROI()